# Seldon V2 Pipeline DataFlow Batching Example


 * Build if needed and place `seldon` binary in your path
    * run `make build-seldon` from operator folder and add bin folder to PATH
 * Run Seldon V2 `make deploy-local` from top level folder
 * Install `kafkacat` for debugging kafka topics & messages

In [ ]:
!which seldon

In [ ]:
!kafkacat -b localhost:9092 -L | grep -i seldon -A1

## Model Batched Chaining based on batch size

In [ ]:
!cat ../../samples/models/sklearn-iris-gs.yaml

In [ ]:
!seldon model load -f ../../samples/models/sklearn-iris-gs.yaml 

In [ ]:
!seldon model status --model-name iris -w ModelAvailable | jq .

In [ ]:
!cat ../../samples/pipelines/iris-batch.yaml

In [ ]:
!seldon pipeline load -f ../../samples/pipelines/iris-batch.yaml

In [ ]:
!seldon pipeline status -p iris-batch -w PipelineReady| jq .

In [ ]:
!seldon pipeline infer -p iris-batch --inference-mode grpc \
    '{"model_name":"iris","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4]},"datatype":"INT32","shape":[1,4]}]}' | jq .

In [ ]:
!kafkacat -b localhost:9092 -t seldon.default.model.iris.outputs -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

In [ ]:
!seldon pipeline infer -p iris-batch --inference-mode grpc \
    '{"model_name":"iris","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4]},"datatype":"INT32","shape":[1,4]}]}' | jq .

In [ ]:
!kafkacat -b localhost:9092 -L | grep -i seldon -A1

In [ ]:
!kafkacat -b localhost:9092 -t seldon.default.pipeline.iris-batch.inputs -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

In [ ]:
!kafkacat -b localhost:9092 -t seldon.default.model.iris.inputs -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

In [ ]:
!kafkacat -b localhost:9092 -t seldon.default.model.iris.outputs -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

### Cleanup

In [ ]:
!seldon pipeline unload -p iris-batch

In [ ]:
!seldon model unload --model-name iris